## import dependencies

In [1]:
from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric, concatenate_datasets, DatasetDict

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

/home/p/miniconda3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/p/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)
PRETRAINED_MODEL_PATH = "google/pegasus-cnn_dailymail"
TOKENIZER = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_PATH)
PRETRAINED_MODEL = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL_PATH).to(DEVICE)

cuda


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
samsum_data = load_dataset("samsum")
samsum_data

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
sam_split_len = [len(samsum_data[split])for split in samsum_data]
samsum_data = samsum_data.remove_columns('id')

In [10]:
def merge_datasets(ds):
  merged_dataset = DatasetDict()
  merged_dataset['train'] = concatenate_datasets([dataset['train'] for dataset in ds],split="train")
  merged_dataset['test'] = concatenate_datasets([dataset['test'] for dataset in ds],split="test")
  merged_dataset['validation'] = concatenate_datasets([dataset['validation'] for dataset in ds if len(dataset)==3])
  return merged_dataset
dataset = merge_datasets([samsum_data])

In [15]:
article = dataset['test'][0]['dialogue']

len(article.split(" ")), article

(57,
 "Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye")

In [16]:
pipe = pipeline('summarization', model=PRETRAINED_MODEL,tokenizer=TOKENIZER,device=DEVICE)

In [17]:
params = {
    "max_length": 128,
    "min_length": 8,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.2,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 4,
}
pipe_out = pipe(article,**params)
print(pipe_out)

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


[{'summary_text': 'Hannah asks Amanda if she can find Betty’s number .'}]
